# optics-matching example - Comparing to Bmad

In [ ]:
import numpy as np
import impact.z as IZ
from impact.z.interfaces.bmad import export_particles as tao_export_particles
from pytao import Tao

from pmd_beamphysics import single_particle
from pmd_beamphysics.units import mec2

from impact.tests.z.conftest import bmad_files

In [ ]:
reference_particle_mass = mec2
energy = 10e6
gamma = energy / mec2
sigma_x0 = 0.001
norm_emit_x = 1e-12
norm_emit_y = 1e-12
beta_x = sigma_x0**2 * gamma / norm_emit_x
beta_y = beta_x

pz = np.sqrt(energy**2 - mec2**2)

# Should give beta_x = 19569511.835591838 m
P0 = single_particle(x=0.0, pz=pz, species="electron")

In [ ]:
!cat $bmad_files/optics_matching.bmad

In [ ]:
tao = Tao(lattice_file=bmad_files / "optics_matching.bmad", plot="mpl")

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
P0.write("drift_initial_particles.h5")
tao.cmds(
    [
        "set beam_init position_file = drift_initial_particles.h5",
        f"set beam_init n_particle = {len(P0)}",
        f"set beam_init bunch_charge = {P0.charge}",
        "set beam_init saved_at = *",
        "set global track_type = single",
        "set global track_type = beam",
    ]
)

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
# TODO write header showing repr as well
input = IZ.ImpactZInput.from_tao(tao)

In [ ]:
input.write_particles_at([], every=IZ.Drift)

In [ ]:
input.write_fulls

In [ ]:
input.lattice

In [ ]:
len(input.lattice)

In [ ]:
# Verify the ParticleGroup we dumped from Tao is the same one we specify as initial_particles
assert input.initial_particles == P0

In [ ]:
input.space_charge_off()
# input.space_charge_on(current=1.0)

In [ ]:
input.current_list = [1e9] * 20
input.particle_list = [1e9] * 20
input.charge_over_mass_list = [1e9] * 20

In [ ]:
I = IZ.ImpactZ(input, use_temp_dir=False, workdir="./tmp", initial_particles=P0)

In [ ]:
I.input.reference_frequency = 1
I.input.n_particle = len(P0)

In [ ]:
output = I.run(verbose=True)

In [ ]:
I.input

In [ ]:
I.output

In [ ]:
Pin = I.output.particles["initial_particles"]

In [ ]:
# Bmad
P0.data

In [ ]:
Pin.data

In [ ]:
# Verify that ImpactZ read and wrote our initial particles correctly:
assert P0 == Pin

In [ ]:
P_raw = I.output.particles_raw["final_particles"]
P = I.output.particles["final_particles"]

In [ ]:
P_raw

In [ ]:
P.data

In [ ]:
Ptao = tao_export_particles(tao, "END")

In [ ]:
Ptao.data

In [ ]:
import matplotlib.pyplot as plt

stats = I.output.stats
z = stats.z
x = stats.mean_x
y = stats.mean_y

x_tao = tao.bunch_comb("x")
y_tao = tao.bunch_comb("y")
s_tao = tao.bunch_comb("s")

fig, (ax0, ax1) = plt.subplots(2, figsize=(8, 4))

ax0.plot(z, x, label="Impact-Z")
ax0.plot(s_tao, x_tao, "--", label="Tao")
ax0.set_ylabel(r"$x$ (m)")

ax1.plot(z, y, label="Impact-Z")
ax1.plot(s_tao, y_tao, "--", label="Tao")
ax1.set_ylabel(r"$y$ (m)")
ax1.set_xlabel(r"$s$ (m)")

plt.legend();